# Toronto Neighbors

* 1. Start by creating a new Notebook for this assignment.
* 2. Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:

In [50]:
import requests
import lxml.html as lh
import pandas as pd

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)

url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = requests.get(url)
doc = lh.fromstring(page.content)
tr_elements = doc.xpath('//tr')

col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print(name)
    if(i==3):
        name = name[:-1]
    col.append((name,[]))

for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #i is the index of our column
    i=0
    if(len(T)!=3):
        break
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        #Append the data to the empty list of the i'th column
        if(i==2):
            data = data[:-1]
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)
print(df)



Postcode
Borough
Neighbourhood

    Postcode           Borough                                     Neighbourhood
0        M1A      Not assigned                                      Not assigned
1        M2A      Not assigned                                      Not assigned
2        M3A        North York                                         Parkwoods
3        M4A        North York                                  Victoria Village
4        M5A  Downtown Toronto                                      Harbourfront
5        M5A  Downtown Toronto                                       Regent Park
6        M6A        North York                                  Lawrence Heights
7        M6A        North York                                    Lawrence Manor
8        M7A      Queen's Park                                      Not assigned
9        M8A      Not assigned                                      Not assigned
10       M9A         Etobicoke                                  Islington Ave

In [53]:
# clean Borough-not assigned
dfc = df.loc[df['Borough']!='Not assigned']

# clean Neighbor-not assigned
def clean_neighbor(row):
    if(row['Neighbourhood']=='Not assigned'):
        row['Neighbourhood']= row['Borough']
    return row

dfc.apply(clean_neighbor, axis=1)

# postcode merge
dfc = dfc.groupby('Postcode').agg({'Borough':'first', 'Neighbourhood':','.join}).reset_index()

# dfc = dfc.reset_index(drop=True)
print(dfc)

    Postcode           Borough                                      Neighbourhood
0        M1B       Scarborough                                      Rouge,Malvern
1        M1C       Scarborough               Highland Creek,Rouge Hill,Port Union
2        M1E       Scarborough                    Guildwood,Morningside,West Hill
3        M1G       Scarborough                                             Woburn
4        M1H       Scarborough                                          Cedarbrae
5        M1J       Scarborough                                Scarborough Village
6        M1K       Scarborough          East Birchmount Park,Ionview,Kennedy Park
7        M1L       Scarborough                      Clairlea,Golden Mile,Oakridge
8        M1M       Scarborough      Cliffcrest,Cliffside,Scarborough Village West
9        M1N       Scarborough                         Birch Cliff,Cliffside West
10       M1P       Scarborough  Dorset Park,Scarborough Town Centre,Wexford He...
11       M1R    

In [54]:
print(dfc.shape)

(103, 3)
